In [2]:
# Load important libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression

from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import Ridge, Lasso

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.svm import LinearSVC
from sklearn.svm import SVC #note that this is non-linear SVM and its default is RBF
from sklearn.datasets import make_regression #function used for generating synthetic datasets
from sklearn.datasets import make_classification, make_blobs

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import mean_squared_error, r2_score


# Import data
train_data = pd.read_excel("./asg1_train_test_data/asg1_traindata.xlsx")
test_data = pd.read_excel("./asg1_train_test_data/asg1_testdata_without_prices.xlsx")

features_X = ["Year", "Type", "Shift", "km", "Power", "Engine", "Seller"]
features_y = ["Price"]


# Split data into features - prices
X_data = train_data[features_X]
y_data = train_data[features_y]


# Split data into train and test sets
# default is 75% / 25% train-test split
# X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state=735)

# X_test = test_data[features_X]
# y_test = test_data[features_y] Y values of test is not given
# y_test = pd.read_csv("./Result.txt", sep=" ", header=None) 

In [3]:
''' 
# Deal with categorical data
from sklearn.preprocessing import OneHotEncoder

# Find the columns to deal with
object_cols = [col for col in X_data.columns if X_data[col].dtype == "object"]

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.fit_transform(X_test[object_cols]))

# Give them column names (for example "Shift_Auto, Shift_Manual") instead of just integer column names (like "0, 1, 2")
OH_cols_train.columns = OH_encoder.get_feature_names_out(object_cols)
OH_cols_test.columns = OH_encoder.get_feature_names_out(object_cols)

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_test.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)
'''



' \n# Deal with categorical data\nfrom sklearn.preprocessing import OneHotEncoder\n\n# Find the columns to deal with\nobject_cols = [col for col in X_data.columns if X_data[col].dtype == "object"]\n\n# Apply one-hot encoder to each column with categorical data\nOH_encoder = OneHotEncoder(handle_unknown=\'ignore\', sparse=False)\nOH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))\nOH_cols_test = pd.DataFrame(OH_encoder.fit_transform(X_test[object_cols]))\n\n# Give them column names (for example "Shift_Auto, Shift_Manual") instead of just integer column names (like "0, 1, 2")\nOH_cols_train.columns = OH_encoder.get_feature_names_out(object_cols)\nOH_cols_test.columns = OH_encoder.get_feature_names_out(object_cols)\n\n# One-hot encoding removed index; put it back\nOH_cols_train.index = X_train.index\nOH_cols_test.index = X_test.index\n\n# Remove categorical columns (will replace with one-hot encoding)\nnum_X_train = X_train.drop(object_cols, axis=1)\nnum_X_test =

In [4]:
# Deal with categorical data
from sklearn.preprocessing import OneHotEncoder

# Find the columns to deal with
object_cols = [col for col in X_data.columns if X_data[col].dtype == "object"]

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(X_data[object_cols]))

# Give them column names (for example "Shift_Auto, Shift_Manual") instead of just integer column names (like "0, 1, 2")
OH_cols.columns = OH_encoder.get_feature_names_out(object_cols)

# One-hot encoding removed index; put it back
OH_cols.index = X_data.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_data.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_data = pd.concat([num_X_train, OH_cols], axis=1)

# Split data into train and test sets
# default is 75% / 25% train-test split
X_train, X_test, y_train, y_test = train_test_split(OH_X_data, y_data, random_state=735)

# Feature scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

##X_train, X_test, y_train, y_test = train_test_split(X_house, y_house)

X_train_scaled = scaler.fit_transform(X_train) # we use X_train to adjust/fit the scaler

X_test_scaled  = scaler.transform(X_test) #use the same fit found above to transform X_test

In [5]:
# Make lineer regression and calculate r-square
from sklearn.linear_model import LinearRegression



# Create linear regression object
linr = LinearRegression()

# Train the model using the training sets
linr.fit(X_train, y_train)

# Make predictions using the testing set
linr_y_pred = linr.predict(X_test)

# linr_mse = mean_squared_error(y_test, linr_y_pred)
linr_r2 = r2_score(y_test, linr_y_pred)

# The coefficients
# print("Coefficients: \n", linr.coef_)

# The coefficient of determination: 1 is perfect prediction
print("R2: %.2f" % linr_r2)
print("R2: %.2f" % linr.score(X_test, y_test))

R2: 0.89
R2: 0.89


In [6]:
# Make ridge regression and calculate r-square
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold


# Define model
ridr = Ridge(alpha=1)

## Fit model with data
ridr.fit(X_train, y_train)

ridr_y_pred = ridr.predict(X_test)

# define model evaluation method
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

ridr_r2 = r2_score(y_test, ridr_y_pred)

# The coefficient of determination: 1 is perfect prediction
print("R2: %.2f" % ridr_r2)
print("R2: %.2f" % ridr.score(X_test, y_test))

R2: 0.89
R2: 0.89


In [7]:
# Make lasso regression and calculate r-square
from sklearn.linear_model import Lasso


# Define model
lasr = Lasso(alpha=10)

## Fit model with data
lasr.fit(X_train, y_train)

lasr_y_pred = lasr.predict(X_test)

# define model evaluation method
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

lasr_r2 = r2_score(y_test, lasr_y_pred)

# The coefficient of determination: 1 is perfect prediction
print("R2: %.2f" % lasr_r2)
print("R2: %.2f" % lasr.score(X_test, y_test))

R2: 0.89
R2: 0.89


In [8]:
# Make KNN regression and calculate r-square
from sklearn.neighbors import KNeighborsClassifier


# Define model
knn = KNeighborsClassifier(n_neighbors = 5).fit(X_train, y_train)

knn_y_pred = knn.predict(X_test)

knn_r2 = r2_score(y_test, knn_y_pred)

# The coefficient of determination: 1 is perfect prediction
print("R2: %.2f" % knn_r2)
print("R2: %.2f" % knn.score(X_test, y_test))


R2: 0.22
R2: 0.02


c:\Users\talha\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\talha\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\Users\talha\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode

In [9]:
# Make SVM
from sklearn.svm import LinearSVC


linsvc = LinearSVC().fit(X_train, y_train)

print('Accuracy of Linear SVC classifier on test set: {:.2f}'.format(linsvc.score(X_test, y_test)))





c:\Users\talha\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of Linear SVC classifier on test set: 0.00


c:\Users\talha\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [10]:
# Make non linear svm
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

svc = SVC(C=0.01).fit(X_train_scaled, y_train)
print(round(svc.score(X_test_scaled, y_test), 3))

c:\Users\talha\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.01
